In [59]:
import pandas as pd
from os import listdir
from os.path import isfile, join
files = ['./data/' + f for f in listdir('./data/') if isfile(join('./data/', f))]

trainData = []
valData = []
testData = []

for i, fileName in enumerate(files):
    if i < 120:
        trainData.append(fileName)
    elif i < 160:
        valData.append(fileName)
    else:
        testData.append(fileName)


df = pd.read_csv(trainData[1])

print(df.iloc[3])

ticker              2000-06-13
ISRG US Equity           2.007
ISRG US Equity.1           2.0
ISRG US Equity.2           NaN
ISRG US Equity.3           NaN
ISRG US Equity.4           NaN
Predictions               DOWN
OneHotPred                   0
Name: 3, dtype: object


In [60]:
import torch
from torch.utils.data import Dataset
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt

class CustomImageDataset(Dataset):
    def __init__(self, files, transform=None, target=None):
        self.lengths = []
        self.data = []

        for f in files:
            df = pd.read_csv(f)
            self.lengths.append(len(df) - 2)
            self.data.append(df)

        self.transforms = transform
        self.target = target

    def __len__(self):
        return sum(self.lengths)

    def __getitem__(self, idx):
        
        i = 0
        while idx > self.lengths[i]:
            idx -= self.lengths
            i += 1
        
        rowsBack = 5
        quiz = torch.zeros(25)
        label = torch.zeros(1)
        quizI = 0

        while rowsBack > 0:

            rowsBack -= 1
            if i < 0:
                i -= 1
                idx = self.lengths[i]
            
            row = self.data[i].iloc[idx + 2]
            for p in range(5):
                quiz[quizI] = row[p + 1]
                quizI += 1

            if rowsBack == 5:
                label = row[7]

        quiz = quiz.div(quiz.norm())

        return quiz, label
        

trainDataset = CustomImageDataset(trainData)
valDataset = CustomImageDataset(valData)
testDataset = CustomImageDataset(testData)
        
        


In [49]:
import torch
k = torch.rand(25)
k[0] = torch.tensor(300)
print(k)
k.div(k.norm())

tensor([3.0000e+02, 2.3775e-01, 6.5520e-01, 3.3088e-01, 4.5856e-01, 4.5968e-01,
        4.0901e-01, 8.6381e-01, 6.0047e-01, 1.8722e-01, 3.2739e-01, 8.3864e-01,
        6.7383e-01, 5.5910e-01, 9.4522e-01, 9.5783e-02, 7.2780e-01, 3.3566e-01,
        5.3283e-01, 6.0120e-01, 1.1613e-02, 3.2479e-01, 8.9843e-01, 3.1312e-01,
        2.7657e-01])


tensor([9.9996e-01, 7.9248e-04, 2.1839e-03, 1.1029e-03, 1.5285e-03, 1.5322e-03,
        1.3633e-03, 2.8793e-03, 2.0015e-03, 6.2403e-04, 1.0912e-03, 2.7953e-03,
        2.2460e-03, 1.8636e-03, 3.1506e-03, 3.1926e-04, 2.4259e-03, 1.1188e-03,
        1.7760e-03, 2.0039e-03, 3.8709e-05, 1.0826e-03, 2.9947e-03, 1.0437e-03,
        9.2187e-04])